# Looking more closely into the crossover examples.
Looking more closely into the crossover events.

In [1]:
%matplotlib inline

In [4]:
#importing necessary modules
import vcfparser
import sys
from operator import itemgetter
import matplotlib.pyplot as plt
import ast
from joblib import Parallel, delayed
import random
import numpy as np
from sklearn.metrics import jaccard_similarity_score

#Loading necessary parameters.
vcfroot = "../../vcf/GATK/"
vcfFileNames = ["1335full_phased.vcf", "1007_phased.vcf", "1012_phased.vcf", "1013_phased.vcf", "1014_phased.vcf", "1015_phased.vcf", "IT_phased.vcf"]
strainIDs = ["1335", "1007", "1012", "1013", "1014", "1015", "3367"]
vcfObjects = []
for i in vcfFileNames:
    vcfObjects.append(vcfparser.VCF(vcfroot+i))    

=======../../vcf/GATK/1335full_phased.vcf=======
#SNPs: 166939
#INDELs: 8634
#VARs: 175573
#HAPBLOCKs: 27724
AVG SNP DEPTH: 78.9326377063
TOTAL LENGTH COVERED: 22884028 bps

=======../../vcf/GATK/1007_phased.vcf=======
#SNPs: 127221
#INDELs: 6114
#VARs: 133335
#HAPBLOCKs: 17129
AVG SNP DEPTH: 31.2347845652
TOTAL LENGTH COVERED: 21074161 bps

=======../../vcf/GATK/1012_phased.vcf=======
#SNPs: 133119
#INDELs: 7099
#VARs: 140218
#HAPBLOCKs: 12206
AVG SNP DEPTH: 55.4267283801
TOTAL LENGTH COVERED: 22834488 bps

=======../../vcf/GATK/1013_phased.vcf=======
#SNPs: 225281
#INDELs: 11471
#VARs: 236752
#HAPBLOCKs: 19570
AVG SNP DEPTH: 49.9257197405
TOTAL LENGTH COVERED: 27558517 bps

=======../../vcf/GATK/1014_phased.vcf=======
#SNPs: 71266
#INDELs: 2285
#VARs: 73551
#HAPBLOCKs: 11309
AVG SNP DEPTH: 26.0904950307
TOTAL LENGTH COVERED: 13439146 bps

=======../../vcf/GATK/1015_phased.vcf=======
#SNPs: 137099
#INDELs: 7461
#VARs: 144560
#HAPBLOCKs: 12428
AVG SNP DEPTH: 50.858065855
TOTAL LENGTH C

In [5]:
def loaddict(filename):
    s = open(filename, 'r').read()
    return eval(s)

# Code for looking at crossovers between two strains

In [6]:
#Compairs two pairs of variants from two strains
#and detects if there is a sign of crossover.
#List of potential outputs

#-2: fatal error happened
#-1: point mutation detected
#0: no crossover detected
#1: crossover detected

def isCrossOver(v0a,v1a,v0b,v1b,verbose=False):
    #first compare and seee if they have the same pairs

    #make sure there is no mutation
    #check for the first position.
    if not(isMutation(v0a.ref, v0a.alt, v0b.ref, v0b.alt)):
        if verbose: print "mutation1"
        return -1
        
    #check for the second position.
    if not(isMutation(v1a.ref, v1a.alt, v1b.ref, v1b.alt)):
        if verbose: print "mutation2"
        return -1

    #determine if there was recombination
    #Comparing only the block-1 not block-2.
    #a1 and a2 both represnt haplotypes for strain A.
    a1 = []
    a2 = []
    if v0a.format["HP"].ref == 1:
        a1.append(v0a.ref)
        a2.append(v0a.alt)
    elif v0a.format["HP"].alt == 1:
        a1.append(v0a.alt)
        a2.append(v0a.ref)
    else:
        print "This should never happen"
        return -2
    
    if v1a.format["HP"].ref == 1:
        a1.append(v1a.ref)
        a2.append(v1a.alt)
    elif v1a.format["HP"].alt == 1:
        a1.append(v1a.alt)
        a2.append(v1a.ref)
    else:
        print "This should never happen"
        return -2
    
    #b1 and b2 both represnt haplotypes for strain B.
    b1 = []
    b2 = []
    if v0b.format["HP"].ref == 1:
        b1.append(v0b.ref)
        b2.append(v0b.alt)
    elif v0b.format["HP"].alt == 1:
        b1.append(v0b.alt)
        b2.append(v0b.ref)
    else:
        print "This should never happen"
        return -2

    if v1b.format["HP"].ref == 1:
        b1.append(v1b.ref)
        b2.append(v1b.alt)
    elif v1b.format["HP"].alt == 1:
        b1.append(v1b.alt)
        b2.append(v1b.ref)
    else:
        print "This should never happen"
        return -2

    #now check if there is a crossover.
    if a1 != b1 and a1 != b2:
        if verbose:
            print
            print a1, b1
            print a2, b2
        
        return 1
    return 0

#This function checks if a point mutation has happend in this pair
#the pair will be disregarded if point mutation is spotted.
def isMutation(x1, x2, y1, y2):
    if x1 == y1 and x2 == y2:
        return True
    elif x1 == y2 and x2 == y1:
        return True
    else:
        return False

In [7]:
def visiualize_crossover(chrom, co, vcf1, vcf2, name1, name2, size=100):
    name1 += " "*(max([len(name1),len(name2)])-len(name1))
    name2 += " "*(max([len(name1),len(name2)])-len(name2))
    
    block1 = [v.pos for v in vcf1.getHapBlockByPos(chrom, co[0])]
    block2 = [v.pos for v in vcf2.getHapBlockByPos(chrom, co[1])]
    
    temp = list(set(block1).union(set(block2)))
    temp.sort()
    
    str1 = ["", ""]
    str2 = ["", ""]
    common = ""
    _id = 1
    
    co_check = []
    for i in range(len(temp)):
        if temp[i] in block1 and temp[i] in block2:
            print str(_id)+":", "POS:", temp[i], "PQ:", vcf1.getVariant(chrom, temp[i]).format["PQ"], vcf2.getVariant(chrom, temp[i]).format["PQ"]
            co_check.append(temp[i])
            if len(str(_id))==1:
                common += str(_id)+" "
            else:
                common += str(_id)
            _id += 1 
        else:
            common += "  "
        if temp[i] in block1:
            blockinfo =  vcf1.getVariant(chrom, temp[i]).format["HP"] 
            str1[blockinfo.ref-1] += vcf1.getVariant(chrom, temp[i]).ref+"-"
            str1[blockinfo.alt-1] += vcf1.getVariant(chrom, temp[i]).alt+"-"
        else:
            str1[0] += "--"
            str1[1] += "--"
        if temp[i] in block2:
            blockinfo =  vcf2.getVariant(chrom, temp[i]).format["HP"]
            str2[blockinfo.ref-1] += vcf2.getVariant(chrom, temp[i]).ref+"-"
            str2[blockinfo.alt-1] += vcf2.getVariant(chrom, temp[i]).alt+"-"
        else:
            str2[0] += "--"
            str2[1] += "--"

    crossover = list(("  "*len(temp))[:-1])
            
    for i in range(len(co_check)-1):
        p = (co_check[i], co_check[i+1])
        v0a = vcf1.getVariant(chrom, p[0])
        v1a = vcf1.getVariant(chrom, p[1])
        v0b = vcf2.getVariant(chrom, p[0])
        v1b = vcf2.getVariant(chrom, p[1])
        if 1 == isCrossOver(v0a,v1a,v0b,v1b):
            i1 = temp.index(p[0]) 
            i2 = temp.index(p[1])
            crossover[i1+i2] = "x"
    
    crossover = "".join(crossover)
    
    for i in range(len(common)/size+1):
        print
        print " "*max([len(name1),len(name2)])+"  "+common[i*size:(i+1)*size]
        print name1+": "+str1[0][:-1][i*size:(i+1)*size]
        print name1+": "+str1[1][:-1][i*size:(i+1)*size]
        print " "*max([len(name1),len(name2)])+"  "+crossover[i*size:(i+1)*size]
        print name2+": "+str2[0][:-1][i*size:(i+1)*size]
        print name2+": "+str2[1][:-1][i*size:(i+1)*size]
        print " "*max([len(name1),len(name2)])+"  "+common[i*size:(i+1)*size]
    
    return (block1[0], block2[0])

In [8]:
def visualize_two_strains(strain1, strain2, chrom, co, vcfFileNames, strainIDs, vcfObjects):
    filebase = vcfFileNames[strain1]+"vs"+vcfFileNames[strain2]
    return visiualize_crossover(chrom, co, vcfObjects[strain1], vcfObjects[strain2], strainIDs[strain1], strainIDs[strain2])

In [9]:
def visiualize_chrom(strain1, strain2, chrom, vcfFileNames, strainIDs, vcfObjects):
    filebase = vcfFileNames[strain1]+"vs"+vcfFileNames[strain2]
    copairs = loaddict(filebase+".copairs")[chrom]
    print "# Crossover on "+chrom+":", len(copairs)
    done = [[],[]]
    _id = 1
    
    for p in copairs:
        if not(vcfObjects[strain1].getHapBlockByPos(chrom, p[0])[0].pos in done[0]) and\
        not(vcfObjects[strain2].getHapBlockByPos(chrom, p[0])[0].pos in done[1]):   
            print "=========================================================="
            print "ID:", _id
            print
            temp = visualize_two_strains(strain1, strain2, chrom, p, vcfFileNames, strainIDs, vcfObjects)
            done[0].append(temp[0])
            done[1].append(temp[1])
            print
            _id += 1
    

# 1335 vs 1007 (within L-clade) on chromosome 1
Observations:
- Crossover does seem to happen more when phasing quality (PQ) is low. Be noted that PQ of a SNP is how well the SNP is phased to the preceeding SNP.
- a pair crossover events found in two consecutive pairs might be false discovery?

Here is GATK ReadBackedPhasing's documentation about PQ score:
- "RBP attempts to phase a heterozygous genotype relative the preceding HETEROZYGOUS genotype for that sample. If there is sufficient read information to deduce the two haplotypes (for that sample), then the current genotype is declared phased ("/" changed to "|") and assigned a PQ that is proportional to the estimated Phred-scaled error rate. All homozygous genotypes for that sample that lie in between the two heterozygous genotypes are also assigned the same PQ value (and remain phased)."

This means that one SNP that has low PQ can completely mess up subsequent SNPs in terms of phasing.

In [10]:
visiualize_chrom(0, 1, "Chr1", vcfFileNames, strainIDs, vcfObjects)

# Crossover on Chr1: 38
ID: 1

1: POS: 44953 PQ: 33.76 44.0
2: POS: 44984 PQ: 140.05 44.0
3: POS: 47102 PQ: 44.69 25.68
4: POS: 47291 PQ: 48.77 103.53
5: POS: 47297 PQ: 2550.05 638.1
6: POS: 47348 PQ: 61.76 58.58
7: POS: 47757 PQ: 61.76 61.76
8: POS: 50198 PQ: 37.8 61.76

                  1 2         3       4 5 6             7               8                                 
1335: --C-C-T-----T-T-----G---T-------C-T-A-------------T---------------A-C-------C-----------------------
1335: --T-T-A-----C-C-----A---G-------T-C-T-------------C---------------G-A-------T-----------------------
                         x        x                                                                       
1007: G-------T-C-T-T-C-T---T-G-A-C-A-C-T-A-A-A-A-T-C-C-T-C-T-G-T-A-C-T-A---A-A-C---A-T-A-C-A-C-T-T-T-C-G-
1007: A-------C-T-C-C-T-C---C-T-G-G-T-T-C-T-T-T-C-C-T-T-C-T-C-A-C-T-T-C-G---G-C-T---T-C-G-T-G-G-A-C-C-A-T-
                  1 2         3       4 5 6             7               8            

# 1335 vs 1015 (within L-clade) on chromosome 1

In [11]:
visiualize_chrom(0, 5, "Chr1", vcfFileNames, strainIDs, vcfObjects)

# Crossover on Chr1: 38
ID: 1

1: POS: 61477 PQ: inf 112.36
2: POS: 66773 PQ: 61.76 29.76
3: POS: 66782 PQ: 1030.63 1804.45
4: POS: 66870 PQ: 53.31 104.5
5: POS: 69005 PQ: 64.77 51.77
6: POS: 69448 PQ: 61.76 56.98
7: POS: 69706 PQ: 56.65 64.77
8: POS: 69728 PQ: 1172.43 237.93
9: POS: 72386 PQ: 60.01 28.31
10: POS: 72396 PQ: 1277.21 1203.48

                                                                                                          
1335: ----------------------------------------------------------------------------------------------------
1335: ----------------------------------------------------------------------------------------------------
                                                                                                          
1015: C-C-T-C-T-G-T-A-C-G-A-A-A-C-A-A-T-C-C-T-A-T-C-C-G-A-C-C-C-G-C-C-T-T-A-A-T-A-C-C-A-A-C-C-T-A-G-C-C-T-
1015: A-T-C-T-C-A-C-G-G-T-G-T-T-T-T-C-C-T-T-C-T-A-T-A-A-G-A-T-A-A-T-A-C-C-G-C-C-G-T-T-G-G-T-A-G-C-T-G-T-A-
               

# 1335 vs IT (L-clade vs H-clade) on chromosome 1

In [12]:
visiualize_chrom(0, 6, "Chr1", vcfFileNames, strainIDs, vcfObjects)

# Crossover on Chr1: 67
ID: 1

1: POS: 31898 PQ: 27.75 32.25
2: POS: 34811 PQ: 1326.05 61.76

                        1                         2   
1335: ----G-------------G-T-C---------------A-C-A-G-G
1335: ----T-------------T-C-A---------------G-A-C-T-C
                                     x               
3367: T-G---T-T-A-T-G-T-T-----C-T-A-A-C-G-T-------G--
3367: A-A---C-G-G-C-A-C-G-----A-A-C-T-T-T-A-------T--
                        1                         2   

ID: 2

1: POS: 43692 PQ: inf 1010.11
2: POS: 43723 PQ: 129.26 621.22

            1   2       
1335: ------A---T------
1335: ------T---G------
              x        
3367: G-T-C-A-A-G-A-G-A
3367: T-C-A-T-G-T-T-C-G
            1   2       

ID: 3

1: POS: 49811 PQ: 756.62 117.89
2: POS: 51918 PQ: 54.36 113.01

                              1             2                       
1335: T-T-A-C-A-C-A-T-C-----C-T-------G---G-G---------------G-C-T--
1335: A-C-G-G-T-T-C-C-T-----T-C-------A---A-C---------------A-A-A--
        

# 1335 vs 1013 (L-clade vs H-clade) on chromosome 1

In [13]:
visiualize_chrom(0, 3, "Chr1", vcfFileNames, strainIDs, vcfObjects)

# Crossover on Chr1: 75
ID: 1

1: POS: 43596 PQ: 863.57 30.75
2: POS: 47102 PQ: 44.69 30.76

            1           2                 
1335: C---C-T-T-T---G---T-C-T-A-T---A-C-C
1335: T---T-A-C-C---A---G-T-C-T-C---G-A-T
                  x                      
1013: --A---T-----T---A-G---------C------
1013: --G---A-----C---C-T---------T------
            1           2                 

ID: 2

1: POS: 73097 PQ: inf 44.44
2: POS: 73112 PQ: 1907.37 564.27
3: POS: 73132 PQ: 651.37 55.25
4: POS: 75296 PQ: 185.03 43.7
5: POS: 75344 PQ: 181.79 35.76

            1 2 3       4     5                       
1335: ------T-A-A-T-----T---A-A---T-A-T-C-T-T-C-G-G-G
1335: ------C-G-G-C-----G---C-C---G-T-A-T-C-A-T-C-T-A
               x    x      x                         
1013: C-A-T-C-G-A---A-G-G-A---A-T--------------------
1013: T-G-C-T-A-G---G-A-T-G---C-C--------------------
            1 2 3       4     5                       

ID: 3

1: POS: 258044 PQ: inf inf
2: POS: 258049 PQ: 1024.01 432.73


# 1013 vs IT (within H-clade) on chromosome 1

In [14]:
visiualize_chrom(3, 6, "Chr1", vcfFileNames, strainIDs, vcfObjects)

# Crossover on Chr1: 128
ID: 1

1: POS: 12507 PQ: 25.31 inf
2: POS: 12519 PQ: 1141.54 1058.44
3: POS: 14374 PQ: 27.75 33.77

                  1 2               3               
1013: T-T-G-A-T-G-A-T-C-A-G-G-A-A-G-C-A-A---A-A-A-C
1013: G-A-A-G-C-T-G-C-A-G-A-A-G-G-A-T-G-T---G-G-C-A
                            x                      
3367: ------------G-C---------------C-----T--------
3367: ------------A-T---------------T-----C--------
                  1 2               3               

ID: 2

1: POS: 26205 PQ: 33.76 35.76
2: POS: 26235 PQ: 854.82 344.99
3: POS: 26246 PQ: 315.49 676.5
4: POS: 26247 PQ: 399.45 851.03
5: POS: 26299 PQ: 57.79 657.47
6: POS: 26310 PQ: 239.63 235.1
7: POS: 31387 PQ: 49.77 61.76

          1   2   3 4     5   6                                                                         7 
1013: --A-A-C-T-T-G-G-----A---G-----A-G---A---A-----A---T-C-A-A---------A-----------------------T-------T-
1013: --T-T-A-A-C-A-A-----G---A-----T-T---G---C-----C---C-T-C-G------

# Questions to address
- How many crossovers happens consecutively?
- What is the phasicing quality of a pair that has a crossover event in between?
- ReadBackedPhasing algorithm paper or may be I need to look into the source code.
- Mitotic recombination process.